In [1]:
import math

In [12]:
# this notebook is for the equilibrium where Kishus give the Umma greeting among themselves
bigX1 = ['empty1', 'blue'] # X1 is signal in dimension 1
bigX2 = ['empty2', 'red'] # X2 is signal in dimension 2
bigY = ['A', 'U', 'L', 'K'] # Y is Kishus action given interaction with any given type of agent
beta = 0.1
proportion_of_akkadians = 0.4 # p_a
proportion_of_ummians = 0.2-(beta/2) # p_u
proportion_of_lagashites = 0.2-(beta/2) # p_l
proportion_of_kishus = 0.2+beta # p_k
# Specifically, this notebook calculates the synergistic information between signals in the first dimension, X1, 
# the second dimension, X2, and the greeting that Kishus give in response to those singals, Y. The calculation of
# synergistic information  is done using the formulation given by Varley and Kaminski (2022)'s "Untangling 
# Synergistic Effects of Intersecting Social Identities with Partial Information Decomposition", using Williams
# and Beer's I_min definition of redundant information. 

In [13]:
# first definining some basic probabilities of signals and actions in the relvant outcome (the mere conjunction)
# then the functions named I... are calculations of mutual information

def P_X1(x, p_a, p_u, p_l, p_k):
    if x == 'empty1':
        ans = p_a+p_l # i.e. the probability that at Lagashite or Akkadian is interacted with, which must be the case for no signal in dimension 1
    elif x == 'blue':
        ans = p_u+p_k
    return ans

def P_X1Y(x, y, p_a, p_u, p_l, p_k):
    if x == 'empty1':
        if y == 'A':
            ans = p_a
        elif y == 'L':
            ans = p_l
        else:
            ans = 0
    elif  x == 'blue':
        if y == 'U':
            ans = p_u+p_k # in the mere conjunction
        else:
            ans = 0
    return ans

def P_Y(y, p_a, p_u, p_l, p_k):
    if y == 'A':
        ans = p_a
    elif y == 'U':
        ans = p_u+p_k # assuming that in the mere conjunction case Kishus give the Umma greeting (calculations are symetric if Lagash instead)
    elif y == 'L':
        ans = p_l
    else:
        ans = 0 # in the merely conjunction equilibrium nobody gives the Kish greeting
    return ans

def P_X2(x, p_a, p_u, p_l, p_k):
    if x == 'empty2':
        ans = p_a+p_u # i.e. the probability that at Ummian or Akkadian is interacted with, which must be the case for no signal in dimension 2
    elif x == 'red':
        ans = p_l+p_k
    return ans

def P_X2Y(x, y, p_a, p_u, p_l, p_k):
    if x == 'empty2':
        if y == 'A':
            ans = p_a
        elif y == 'U':
            ans = p_u
        else:
            ans = 0
    elif  x == 'red':
        if y == 'L':
            ans = p_l
        elif y == 'U': # note that Kishus are giving the Umma greeting among themselves
            ans = p_k
        else:
            ans = 0
    return ans

def P_X1X2(x1, x2, p_a, p_u, p_l, p_k):
    if x1 == 'empty1' and x2 == 'empty2':
        ans = p_a
    elif x1 == 'blue' and x2 == 'red':
        ans = p_k
    else:
        ans = p_u # Ummians, Lagashites and Kishus are always the same proportion of the population on our parameters
    return ans

def P_X1X2Y(x1, x2, y, p_a, p_u, p_l, p_k):
    if x1 == 'empty1' and x2 == 'empty2':
        if y == 'A':
            ans = p_a
        else:
            ans = 0 # Akkadians are only ones who signal empty empty and they always give the Akkadian greeting
    elif x1 == 'blue' and x2 == 'empty2':
        if y == 'U':
            ans = p_u
        else:
            ans = 0
    elif x1 == 'empty1' and x2 == 'red':
        if y == 'L':
            ans = p_l
        else:
            ans = 0
    elif x1 == 'blue' and x2 == 'red':
        if y == 'U': # note that Kishus are giving the Umma greeting among themselves
            ans = p_k
        else:
            ans = 0
    return ans

def I_XY(P_X, P_bigY, P_XY, bigX, bigYp, p_a, p_u, p_l, p_k):
    ans = 0
    for x in bigX:
        for y in bigYp:
            if P_XY(x, y, p_a, p_u, p_l, p_k) != 0:
                ans += P_XY(x, y, p_a, p_u, p_l, p_k)*math.log((P_XY(x, y, p_a, p_u, p_l, p_k))/(P_X(x, p_a, p_u, p_l, p_k)*P_Y(y, p_a, p_u, p_l, p_k)))
    return ans

def I_XXY(P_XX, P_bigY, P_XXY, bigXa, bigXb, bigYp, p_a, p_u, p_l, p_k):
    ans = 0
    for xa in bigXa:
        for xb in bigXb:
            for y in bigYp:
                if P_XXY(xa, xb, y, p_a, p_u, p_l, p_k) != 0:
                    ans +=  P_XXY(xa, xb, y, p_a, p_u, p_l, p_k)*math.log((P_XXY(xa, xb, y, p_a, p_u, p_l, p_k))/(P_XX(xa, xb, p_a, p_u, p_l, p_k)*P_Y(y, p_a, p_u, p_l, p_k)))
    return ans

def Redundant_X1X2Y(rI_XY, rP_Xa, rP_Xb, rP_Y, rP_XaY, rP_XbY, rbigXa, rbigXb, rbigY, p_a, p_u, p_l, p_k): # Williams and Beer's I_min defintion of redundant information
    ans = 0
    for y in rbigY:
        IaXY = rI_XY(rP_Xa, rP_Y, rP_XaY, rbigXa, rbigY, p_a, p_u, p_l, p_k)
        IbXY = rI_XY(rP_Xb, rP_Y, rP_XbY, rbigXb, rbigY, p_a, p_u, p_l, p_k)
        ans += rP_Y(y, p_a, p_u, p_l, p_k)*min(IaXY, IbXY)
    return ans

In [14]:

mutual_infoX1Y = I_XY(P_X1, P_Y, P_X1Y, bigX1, bigY, proportion_of_akkadians, proportion_of_ummians, proportion_of_lagashites, proportion_of_kishus)
print(f'I(X1 ; Y) = {mutual_infoX1Y}')
print(' ')

mutual_infoX2Y = I_XY(P_X2, P_Y, P_X2Y, bigX2, bigY, proportion_of_akkadians, proportion_of_ummians, proportion_of_lagashites, proportion_of_kishus)
print(f'I(X2 ; Y) = {mutual_infoX2Y}')
print(' ')

redundant_info = Redundant_X1X2Y(I_XY, P_X1, P_X2, P_Y, P_X1Y, P_X2Y, bigX1, bigX2, bigY, proportion_of_akkadians, proportion_of_ummians, proportion_of_lagashites, proportion_of_kishus)
print(f'redundant information = {redundant_info}')
print(' ')

UnqX1YX2 = mutual_infoX1Y - redundant_info
UnqX2YX1 = mutual_infoX2Y - redundant_info

mutual_macro = I_XXY(P_X1X2, P_Y, P_X1X2Y, bigX1, bigX2, bigY, proportion_of_akkadians, proportion_of_ummians, proportion_of_lagashites, proportion_of_kishus)
print(f'I(X1, X2 ; Y) = {mutual_macro}')
print(' ')

synergistic_information = mutual_macro - redundant_info - UnqX1YX2 - UnqX2YX1
print(f'synergistic information = {synergistic_information}')
print(' ')

I(X1 ; Y) = 0.6881388137135884
 
I(X2 ; Y) = 0.40170743798092257
 
redundant information = 0.4017074379809226
 
I(X1, X2 ; Y) = 1.0104127537805414
 
synergistic information = 0.322273940066953
 


In [15]:
print(f'coherency check = {redundant_info+UnqX1YX2+UnqX2YX1+synergistic_information}')

coherency check = 1.0104127537805414


In [16]:
print(f'proportion of total information that is redundant = {redundant_info/mutual_macro}')
print(' ')
print(f'proportion of total information that is synergistic = {synergistic_information/mutual_macro}')

proportion of total information that is redundant = 0.3975676637868055
 
proportion of total information that is synergistic = 0.31895276347328244
